# Markov Decision Process

In [1]:
import torch
import itertools
import random

from envs import ObservableEnv, env_gridworld
import rl.mdp as mdp

## Slides Example

### Make Environment as torch tensor

* Reward:
    A 2D array of the rewards of $[s, a]$

* Transition:
    A 3D array of the probability of reaching state $s'$ under state $s$ and taking aciton $a$
    
* Value:
    A 1D array of the expected value at state $s$

In [2]:
states = ['PU', 'PF', 'RU', 'RF']
actions = ['S', 'A']
rewards = torch.tensor([[0, 0],
                        [0, 0],
                        [10, 10],
                        [10, 10]], dtype=torch.float32)
transitions = torch.tensor([[[1, 0, 0, 0], [0.5, 0.5, 0, 0]],
                            [[0.5, 0., 0., 0.5], [0., 1., 0., 0.]],
                            [[0.5, 0., 0.5, 0.], [0.5, 0.5, 0., 0.]],
                            [[0., 0., 0.5, 0.5], [0., 1., 0., 0.]]], dtype=torch.float32)
gamma = 0.9

In [3]:
env = ObservableEnv(states, actions, transitions, rewards)

### Value Iteration

In [4]:
value = torch.tensor([0, 0, 10, 10], dtype=torch.float32)

value_iteration = mdp.ValueIteration(env, value, gamma)
value_iteration.run()

Converged in 90 iterations.


(tensor([31.5824, 38.6013, 44.0214, 54.1988]), tensor([1, 0, 0, 0]))

### Policy Iteration

In [5]:
value = torch.tensor([0, 0, 10, 10], dtype=torch.float32)
policy = torch.tensor([random.randint(0, 1) for _ in range(4)])

policy_iteration = mdp.PolicyIteration(env, value, policy, gamma)
policy_iteration.run()

Converged in 2 iterations.


(tensor([ 0.0000,  4.5000, 14.5000, 19.0000]), tensor([1, 0, 0, 0]))

### Modified Policy Iteration

In [6]:
value = torch.tensor([0, 0, 10, 10], dtype=torch.float32)
policy = torch.tensor([random.randint(0, 1) for _ in range(4)])
n_evals = 5

modified_policy_iteration = mdp.ModifiedPolicyIteration(env, value, policy, gamma, n_evals)
modified_policy_iteration.run()

Converged in 19 iterations.


(tensor([31.5848, 38.6037, 44.0239, 54.2013]), tensor([1, 0, 0, 0]))

## Assigment 1 Part 1

### Make Environment

Grid world layout:

  \---------------------  
  |  0 |  1 |  2 |  3 |  
  \---------------------  
  |  4 |  5 |  6 |  7 |  
  \---------------------  
  |  8 |  9 | 10 | 11 |  
  \---------------------  
  | 12 | 13 | 14 | 15 |  
  \---------------------  

  Goal state: 15   
  Bad state: 9  
  End state: 16  
 
$|S| = 17$  
$|A| = 4$

transitions: $|S| * |A| * |S'|$  
rewards = $|S| * |A|$

In [7]:
states = [str(i) for i in range(17)]
actions = ['up', 'down', 'left', 'right']
transitions = env_gridworld.transitions
rewards = env_gridworld.rewards

gamma = 0.95

In [8]:
env = ObservableEnv(states, actions, transitions, rewards)

### Value Iteration

In [9]:
value = torch.empty(len(states), dtype=torch.float32)

value_iteration = mdp.ValueIteration(env, value, gamma)
value_iteration.run()

Converged in 738 iterations.


(tensor([ 3.9667e+01,  4.6420e+01,  5.4069e+01,  6.1717e+01,  4.0832e+01,
          4.7916e+01,  6.2967e+01,  7.2633e+01,  4.1804e+01, -6.6523e+00,
          7.3775e+01,  8.5318e+01,  5.5056e+01,  6.5748e+01,  8.5318e+01,
          1.0000e+02,  1.1978e-11]),
 tensor([3, 3, 1, 1, 3, 3, 1, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3]))

### Policy Iteration

In [10]:
value = torch.empty(len(states), dtype=torch.float32)
policy = torch.tensor([random.randint(0, len(actions) - 1) for _ in range(len(states))], dtype=torch.int64)

policy_iteration = mdp.PolicyIteration(env, value, policy, gamma)
policy_iteration.run()

Converged in 54 iterations.


(tensor([3.0285e+30, 3.0285e+30, 3.0285e+30, 3.0285e+30, 3.0285e+30, 3.0285e+30,
         3.0285e+30, 3.0285e+30, 3.0285e+30, 3.0285e+30, 3.0285e+30, 3.0285e+30,
         3.0285e+30, 3.0285e+30, 3.0285e+30, 1.0000e+02, 2.6930e-06]),
 tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 2, 3, 3]))

### Modified Policy Iteration

In [11]:
value = torch.empty(len(states))
policy = torch.tensor([random.randint(0, len(actions) - 1) for _ in range(len(states))])
n_evals = 5

modified_policy_iteration = mdp.ModifiedPolicyIteration(env, value, policy, gamma, n_evals)
modified_policy_iteration.run()

Converged in 7 iterations.


(tensor([ 60.6326,  66.0390,  71.8062,  77.0930,  59.8195,  65.1846,  77.8315,
          84.1415,  58.0956,   7.9886,  84.8673,  91.7816,  69.4968,  76.8099,
          91.7816, 100.0000,   0.0000]),
 tensor([3, 3, 3, 1, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3]))

# Reinforcement Learning

## Monte Carlo Evaluation

In [22]:
from envs.env_gridworld import EnvGridWorld
import collections

In [36]:
def MC_evaluation(env, gamma=0.95):
    policy = lambda state: random.randint(0, env.n_actions() - 1)
    value = torch.zeros(env.n_states())
    
    counter = collections.Counter()
    dones = set()
    
    for i in itertools.count(1):
        state = env.reset()
        done = False
        factor = 1
        G = 0
        while not done:
            action = policy(state)
            next_state, reward, done, _ = env.step(action)
            
            counter[state] += 1
            alpha = 1 / counter[state]
            G += factor * reward
            next_value = value[state] + alpha * (G - value[state])
            factor *= gamma
            
            if abs(next_value - value[state]) < 1e-3:
                dones.add(state)
            value[state] = next_value
            if len(dones) == env.n_states() - 1:
                print(f'converged in {i} rounds')
                return value
            state = next_state
        if i % 10000 == 0:
            print(f'{i} rounds has been processed, {len(dones)} are converged')
        
        if i >= 100000:
            print(value)
            print(dones)
            break

### MC Evaluation on maze

In [37]:
%%time
env = EnvGridWorld()
value = MC_evaluation(env,0.999)
print(value)

converged in 771 rounds
tensor([-140.9322, -182.0999, -218.0852, -238.9838, -195.9595, -232.1794,
        -245.0768, -253.5311, -249.8367, -325.9745, -273.7172, -256.5232,
        -248.8553, -287.7272, -286.9494, -182.4487,    0.0000])
CPU times: user 4.87 s, sys: 48 µs, total: 4.87 s
Wall time: 4.87 s


## TD Evalutaion

In [24]:
def TD_evaluation(env, gamma=0.95):
    policy = lambda state: random.randint(0, env.n_actions() - 1)
    value = torch.zeros(env.n_states())
    
    counter = collections.Counter()
    dones = set()
    
    for i in itertools.count(1):
        state = env.reset()
        done = False
        while not done:
            action = policy(state)
            next_state, reward, done, _ = env.step(action)
            
            counter[state] += 1
            next_value = value[state] + (reward + gamma * value[next_state] - value[state]) / counter[state]
            if abs(next_value - value[state]) < 1e-3:
                dones.add(state)
            value[state] = next_value
            if len(dones) == env.n_states() - 1:
                print(f'converged in {i} rounds')
                return value
            state = next_state
        if i % 10000 == 0:
            print(f'{i} rounds has been processed, {len(dones)} are converged')
        
        if i >= 100000:
            print(value)
            print(dones)
            break

### TD evaluation on maze

In [26]:
%%time
env = EnvGridWorld()
value = TD_evaluation(env)
print(value)

converged in 1710 rounds
tensor([ -21.5537,  -21.4420,  -12.0640,   -4.1311,  -33.1369,  -40.2014,
         -15.1712,    2.4095,  -52.3734, -103.5385,  -17.7937,   26.8511,
         -38.6718,  -40.2930,   13.0381,  100.0000,    0.0000])
CPU times: user 12.7 s, sys: 1.1 ms, total: 12.7 s
Wall time: 12.7 s


## TD control (a.k.a. Q-learning)

Q-learning is an alias of TD control algorithm, as we are calling the state-action function as q-function

### Q-learning on maze



In [38]:
def q_learning(env, gamma=0.95):
    policy = lambda value, state: torch.argmax(value[state]).item() if random.random() < 0.9 else random.randint(0, env.n_actions() - 1)
    value = torch.zeros(env.n_states(), env.n_actions())
    
    counter = collections.Counter()
    dones = set()
    
    for i in itertools.count(1):
        state = env.reset()
        done = False
        
        while not done:
            action = policy(value, state)
            next_state, reward, done, _ = env.step(action)
            
            counter[(state, action)] += 1
            alpha = 1 / counter[(state, action)]
            next_value = value[state, action] + alpha * (reward + gamma * torch.max(value[next_state]).item() - value[state, action])
            if abs(next_value - value[state, action]) < 1e-3:
                dones.add((state, action))
            value[state, action] = next_value
            if len(dones) == (env.n_states() - 1) * env.n_actions():
                print(f'converged in {i} rounds')
                return value
            state = next_state
        if i % 10000 == 0:
            print(f'{i} rounds has been processed, {len(dones)} are converged')
        
        if i >= 50000:
            print(value)
            print(dones)
            break

In [39]:
%%time
env = EnvGridWorld()
value = q_learning(env)
print(value)

10000 rounds has been processed, 29 are converged
20000 rounds has been processed, 31 are converged
30000 rounds has been processed, 33 are converged
40000 rounds has been processed, 34 are converged
50000 rounds has been processed, 35 are converged
tensor([[ 4.5484e+01,  3.7568e+01,  4.4004e+01,  5.2755e+01],
        [ 5.3951e+01,  5.4434e+01,  4.8159e+01,  6.0635e+01],
        [ 6.1952e+01,  6.8369e+01,  5.6413e+01,  6.4404e+01],
        [ 6.4771e+01,  7.4762e+01,  6.2408e+01,  4.6644e+01],
        [ 4.0088e+01,  4.3283e+01,  4.0675e+01,  4.9019e+01],
        [ 5.1920e+01,  1.2463e+01,  3.9142e+01,  6.0513e+01],
        [ 6.2222e+01,  7.5964e+01,  5.7032e+01,  7.5491e+01],
        [ 6.8240e+01,  8.3358e+01,  7.0596e+01,  7.6547e+01],
        [ 3.0953e+01,  5.1495e+01,  3.6134e+01,  9.4052e+00],
        [-1.3430e+01,  8.7110e-02, -2.2170e+01,  5.3969e+00],
        [ 6.3412e+01,  7.5727e+01,  1.6713e+01,  8.4333e+01],
        [ 7.8293e+01,  9.1660e+01,  7.9791e+01,  8.5845e+01],
      